<a href="https://colab.research.google.com/github/comet-toolkit/comet_training/blob/main/LPS%20Training%20-%20Exercise%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**LPS hands-on training session: CoMet Toolkit: Uncertainties made easy**

#Exercise 2: Multi-Dimension Datasets

## Objectives

In this exercise we will cover:

* How to use [obsarray](https://obsarray.readthedocs.io/en/latest/) to store error-correlation information for multi-dimensional measurement datasets - such as from Earth Observation.
* Propagating uncertainties from these datasets through measurement functions using [punpy](https://punpy.readthedocs.io/en/latest/).

## Step 1 - Environment Setup

As in Exercise 1, we start by install and importing the required CoMet Toolkit packages --- [obsarray](https://obsarray.readthedocs.io/en/latest/) and [punpy](https://punpy.readthedocs.io/en/latest/).

In [ ]:
# Install & import CoMet Toolkit packages
!pip install obsarray>=1.0.1
!pip install punpy>=1.0.4

import obsarray
import punpy
import xarray as xr

CoMet Toolkit's [**obsarray**](https://obsarray.readthedocs.io/en/latest/) package is an extension to the popular [xarray](https://docs.xarray.dev/en/stable/) package.

[`xarray.Dataset`](https://docs.xarray.dev/en/stable/user-guide/data-structures.html#dataset)'s are objects in python that represent of the data model from the [netCDF file format](https://www.unidata.ucar.edu/software/netcdf/). **obsarray** allows you to assign uncertainties to variables in xarray Datasets, with their associated error-correlation.

This is achieved by using the CoMet Toolkit's [UNC Specification](https://comet-toolkit.github.io/unc_website/) metadata standard for uncertainty. So such objects are portable, and can be stored to and from disc.


## Step 2 - Interacting with a Measurement Dataset using obsarray

In this step of the exercise, we will explore how to define and interact with the *uncertainty variables* associated with *observation variables* in measurement datasets using obsarray.

Our example will be a multi-spectral dataset of Level 1 [Brightness Temperatures](https://en.wikipedia.org/wiki/Brightness_temperature) from the AVHRR sensor on MetOp-A. This [dataset](https://catalogue.ceda.ac.uk/uuid/14a8098d70714cc1bf38f9dbcb82e5ed/) was created as part of the [FIDUCEO](https://research.reading.ac.uk/fiduceo/) project.

Here we open an extract from [netCDF](https://www.unidata.ucar.edu/software/netcdf/), which has an observation variable with the following dimensions:

* $x$, along track -- 400 pixels at 1 km resolution
* $y$, across track -- 400 pixels at 1 km resolution
* $band$, spectral bands -- 2 bands centred on $\sim$11 and 12 $\mu$m


In [ ]:
# open xarray.Dataset from netCDF file
dataset_path = "avhrr_dataset.nc"
avhrr_ds = xr.open_dataset(dataset_path)

# inspect dataset
print("Dataset Variables: " + str(avhrr_ds.keys())
avhrr_ds["brightness_temperature"][0].plot()